# Tobacco Consumption Data Exploration

By Edson Castañeda

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import math

In [ ]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

## Read Data

In [ ]:
DATA_PATH = "../../data/Tobacco_Consumption.csv"
tobacco_data = pd.read_csv(DATA_PATH)
tobacco_data.sample(10)

## Exploration

In [ ]:
tobacco_data.shape

In [ ]:
tobacco_data.info()

### Categorical Data

In [ ]:
tobacco_categorical_data = tobacco_data.select_dtypes(exclude=['int', 'float'])
print(f"Total Categorical Columns: {len(tobacco_categorical_data.columns)}")
tobacco_categorical_data.head(10)

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(20, 15))
fig.subplots_adjust(hspace=.5)
i = 0
for col in tobacco_categorical_data.columns:
    sns.countplot(tobacco_categorical_data[col], ax=ax[i%2, math.floor(i/2)])
    i+=1

for ax in fig.axes:
    plt.sca(ax)
    plt.xticks(rotation=45)


For *LocationDesc* and *LocationAbbrev* columns there is only one unique value each. Therefore, these columns are constants.

#### Categorical data combinations

In [ ]:
tobacco_categorical_data.drop_duplicates()

In [ ]:
total_data = len(tobacco_categorical_data)
different_combinations = len(tobacco_categorical_data.drop_duplicates())
print(f"Total combinations of categories (rows): {total_data}")
print(f"Find {different_combinations} unique category combinations")
print(f"Relation: {total_data/different_combinations}")

In [ ]:
len(tobacco_data['Year'].unique())

The categories in the dataframe are *repeated by year*. Each year has the same 13 combinations of categories.

### Numerical Data

In [ ]:
tobacco_numerical_data = tobacco_data.select_dtypes(include=['int', 'float'])
print(f"Total Numerical Columns: {len(tobacco_numerical_data.columns)}")
tobacco_numerical_data.head(10)

In [ ]:
# Explore correlations
correlations = tobacco_numerical_data.corr()
sns.heatmap(correlations, annot=True)
plt.show()

*Per capita values* have a strong correlation with normal values. 

In [ ]:
relation_per_capita = round(tobacco_numerical_data["Total"]/tobacco_numerical_data["Population"], 1) - tobacco_numerical_data["Total Per Capita"]
round(relation_per_capita.median(), 3)


Per_capita_columns = (Normal_column)/(Population)

For further analysis, per capita columns are excluded.

*Domestic* and *Imports* have a strong correlation to *Total* column.

In [ ]:
difference_total = tobacco_numerical_data["Total"]- tobacco_numerical_data["Domestic"] - tobacco_numerical_data["Imports"]
difference_total.median()

Total = Imports + Domestic

*Year* and *Population* have a strong correation with each other, but a low correation to tobacco values.

### Measures and submesures

As each year has the same category combinations, one year (2000) is used as a sample.

In [ ]:
pounds_df = tobacco_data[(tobacco_data["Data Value Unit"] == "Pounds") & (tobacco_data["Year"] == 2000)]
pounds_df

In [ ]:
# Compare diff between Total Loose Tobacco and Pipe Tobacco
pounds_df["Domestic"][4] - pounds_df["Domestic"][6]
# Diff is equal to Roll-Your-Own Tobacco

For Loose Tobacco (in Pounds):

Total Loose Tobacco = Pipe Tobacco + Roll-Your-Own Tobacco

In [ ]:
cigarette_equivalents_df = tobacco_data[(tobacco_data["Data Value Unit"] != "Pounds") & (tobacco_data["Year"] == 2000)]
cigarette_equivalents_df

For Loose Tobacco (in Cigarette Equivalents):

Total Loose Tobacco = Pipe Tobacco + Roll-Your-Own Tobacco

The Loose Tobacco values are in the table twice (as pounds and as cigarette equivalents), that's the reason the frecuency was the double than other cases in categorical data analysis.

In [ ]:
cigars_df = cigarette_equivalents_df[(cigarette_equivalents_df["Data Value Unit"] == "Cigars")]
cigars_df

In [ ]:
# Add small and large cigars
cigars_df["Domestic"][5] + cigars_df["Domestic"][9]
# Value equal to total cigars

For Cigars:

Total Cigars = Small Cigars + Large Cigars

In [ ]:
# Sum all non-total cigarrette and cigars columns
sum_cigarettes = cigarette_equivalents_df["Domestic"][~cigarette_equivalents_df["Submeasure"].str.contains("Total")].sum()

cigarette_equivalents_df["Domestic"][cigarette_equivalents_df["Submeasure"]=="Total Combustible Tobacco"] - sum_cigarettes
# Sum is equal to Total Combustible Tobacco

Total Combustible Tobacco = Small Cigars + Large Cigars + Pipe Tobacco + Roll-Your-Own Tobacco + Cigarette Removals

Cigarette, Cigarette Equivalents, and Cigars units have a 1:1:1 relationship.

## Transform Data

In [ ]:
# Drop columns with constant information
tobacco_data.drop(columns=["LocationAbbrev", "LocationDesc"], inplace=True)
# Measures and Topics can be grouped due their different units
tobacco_data.drop(columns=["Topic", "Measure"], inplace=True)
# To reduce data with similar behavior, per capita values will be also ignored in the transformation
tobacco_data.drop(columns=["Domestic Per Capita", "Imports Per Capita", "Total Per Capita"], inplace=True)
tobacco_data.head(3)

In [ ]:
# Get all non-total measures
tobacco_singles_df = tobacco_data[~tobacco_data["Submeasure"].str.contains("Total")]
tobacco_singles_df.reset_index(drop=True, inplace=True)
print(f"Shape: {tobacco_singles_df.shape}")
tobacco_singles_df.head(5)


In [ ]:
# Convert cigarettes, cigarettes equivalents and cigars to standard unit "Units"
tobacco_singles_df.loc[tobacco_singles_df["Data Value Unit"] != "Pounds", "Data Value Unit"] = "Units"
tobacco_singles_df.head(5)

In [ ]:
# Combine submasure and data value units
tobacco_singles_df["Type"] = tobacco_singles_df["Submeasure"] + " (" + tobacco_singles_df["Data Value Unit"] + ")"
# Drop columns
tobacco_singles_df.drop(columns=["Submeasure", "Data Value Unit"], inplace=True)
tobacco_singles_df.head(5)

In [ ]:
ts_df =pd.pivot_table(tobacco_singles_df, index = ["Year", "Population"], columns = ["Type"])
ts_df.reset_index(inplace=True)
ts_df.columns = [" ".join(col).strip() for col in ts_df.columns.values]
ts_df.head(5)

In [ ]:
# Add total column
ts_df["Total Combustible Tobacco (Units)"] = (ts_df["Total Cigarette Removals (Units)"] + ts_df["Total Small Cigars (Units)"] +
    ts_df["Total Large Cigars (Units)"] + ts_df["Total Pipe Tobacco (Units)"] + ts_df["Total Roll-Your-Own Tobacco (Units)"])

In [ ]:
# Plot total over the years
sns.lineplot(x=ts_df["Year"], y=ts_df["Total Combustible Tobacco (Units)"])

In [ ]:
# Export ts to df
OUTPUT_PATH = "../../data/Transformed_Tobacco_Consumption.csv"
ts_df.to_csv(OUTPUT_PATH, index=False)

In [ ]:
# Explore variables distribution
ts_df.describe().convert_dtypes()

In [ ]:
# Create boxplots
COLORS = ["b","g", "r", "c", "m", "y"]
fig, ax = plt.subplots(5,6, figsize=(20,15))
i = 0
for col in ts_df.columns:
    sns.boxplot(y=col, data=ts_df, color = random.choice(COLORS), ax=ax[i%5, math.floor(i/5)])
    i+=1

plt.tight_layout()

In [ ]:
# Create histogram
COLORS = ["b","g", "r", "c", "m", "y"]
fig, ax = plt.subplots(5,6, figsize=(20,15))
i = 0
for col in ts_df.columns:
    sns.distplot(ts_df[col], color = random.choice(COLORS), ax=ax[i%5, math.floor(i/5)])
    i+=1

plt.tight_layout()

In [ ]:
# Plot trend by year
COLORS = ["b","g", "r", "c", "m", "y"]
fig, ax = plt.subplots(5,6, figsize=(20,15))
i = 0
for col in ts_df.columns:
    sns.lineplot(x=ts_df["Year"], y=ts_df[col], color = random.choice(COLORS), ax=ax[i%5, math.floor(i/5)])
    i+=1
plt.tight_layout()

In [ ]:
# Explore correlations
correlations = ts_df.corr()
plt.figure(figsize=(20, 15))
sns.heatmap(correlations, annot=True)
plt.show()

In [ ]:
ts_change_df = ts_df.pct_change().dropna()
ts_change_df = round(ts_change_df *100,2)
ts_change_df.drop(columns = ["Year", "Population"], inplace=True)
ts_change_df.head(5)

In [ ]:
# Plot % of change of varibles
COLORS = ["b","g", "r", "c", "m", "y"]
fig, ax = plt.subplots(5,6, figsize=(20,15))
i = 0
for col in ts_change_df.columns:
    sns.lineplot(x=ts_change_df.index, y=ts_change_df[col], color = random.choice(COLORS), ax=ax[i%5, math.floor(i/5)])
    i+=1
plt.tight_layout()